In [112]:
import yfinance as yf

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

In [113]:
# Comments on Tickers adj for Yahoo Finance

#0700.HK, listed on Listed on HKG (Hong Kong Stock Exchange), traded in HKD
#VLA.PA, listed on Listed on EPA (Euronext Paris), traded in EUR
#RR.L,  Listed on LON (London Stock Exchange), traded in GBP
#CTC-A.TO, Listed on TSE (Toronto Stock Exchange), traded in CAD
#SCAN.V, Listed on TSXV (Toronto Stock Exchange), traded in CAD

In [114]:
POSITIONS =[ 
        {'name': 'Apoorva', 'ticker': 'TSM', 'exchange':'NYSE', 'weight': 0.4},
        {'name': 'Apoorva', 'ticker': 'GOOGL', 'exchange':'NASDAQ', 'weight': 0.1},
        {'name': 'Apoorva', 'ticker': 'TAK', 'exchange':'NYSE', 'weight': 0.25},
        {'name': 'Apoorva', 'ticker': '0700', 'exchange':'HKG', 'weight': 0.25},

        {'name': 'Alessandro', 'ticker': 'DUOL', 'exchange':'NASDAQ', 'weight': 0.4},
        {'name': 'Alessandro', 'ticker': 'GOOG', 'exchange':'NASDAQ', 'weight': 0.25},
        {'name': 'Alessandro', 'ticker': 'ZG', 'exchange':'NASDAQ', 'weight': 0.35},

        {'name': 'Ivan', 'ticker': 'VST', 'exchange':'NYSE', 'weight': 0.4},
        {'name': 'Ivan', 'ticker': 'EXPE', 'exchange':'NASDAQ', 'weight': 0.3},
        {'name': 'Ivan', 'ticker': 'DGX', 'exchange':'NYSE', 'weight': 0.3},

        {'name': 'Abhi', 'ticker': 'ASR', 'exchange':'NYSE', 'weight': 0.25},
        {'name': 'Abhi', 'ticker': '0700', 'exchange':'HKG', 'weight': 0.25},
        {'name': 'Abhi', 'ticker': 'SMCI', 'exchange':'NASDAQ', 'weight': 0.25},
        {'name': 'Abhi', 'ticker': 'BRO', 'exchange':'NASDAQ', 'weight': 0.25},
        
        {'name': 'Conor', 'ticker': 'PFE', 'exchange':'NYSE', 'weight': 0.2},
        {'name': 'Conor', 'ticker': 'RR', 'exchange':'LON', 'weight': 0.2},
        {'name': 'Conor', 'ticker': 'MCD', 'exchange':'NYSE', 'weight': 0.2},
        {'name': 'Conor', 'ticker': 'SHEL', 'exchange':'LON', 'weight': 0.2},
        {'name': 'Conor', 'ticker': 'ALV', 'exchange':'XETRA', 'weight': 0.2},
        
        {'name': 'Diarbhail', 'ticker': 'FLUT', 'exchange':'NYSE', 'weight': 0.34},
        {'name': 'Diarbhail', 'ticker': 'NVDA', 'exchange':'NASDAQ', 'weight': 0.33},
        {'name': 'Diarbhail', 'ticker': 'SCAN', 'exchange':'CVE', 'weight': 0.33},

        {'name': 'Radu', 'ticker': 'MP', 'exchange':'NYSE', 'weight': 0.1},
        {'name': 'Radu', 'ticker': 'DBK', 'exchange':'XETRA', 'weight': 0.5},
        {'name': 'Radu', 'ticker': 'RTX', 'exchange':'NYSE', 'weight': 0.2},
        {'name': 'Radu', 'ticker': 'RHM', 'exchange':'XETRA', 'weight': 0.2},

        {'name': 'Silvia', 'ticker': 'NVO', 'exchange':'NYSE', 'weight': 0.33},
        {'name': 'Silvia', 'ticker': 'NVDA', 'exchange':'NASDAQ', 'weight': 0.33},
        {'name': 'Silvia', 'ticker': 'OPRA', 'exchange':'NASDAQ', 'weight': 0.34},
]
POSITIONS

[{'name': 'Apoorva', 'ticker': 'TSM', 'exchange': 'NYSE', 'weight': 0.4},
 {'name': 'Apoorva', 'ticker': 'GOOGL', 'exchange': 'NASDAQ', 'weight': 0.1},
 {'name': 'Apoorva', 'ticker': 'TAK', 'exchange': 'NYSE', 'weight': 0.25},
 {'name': 'Apoorva', 'ticker': '0700', 'exchange': 'HKG', 'weight': 0.25},
 {'name': 'Alessandro', 'ticker': 'DUOL', 'exchange': 'NASDAQ', 'weight': 0.4},
 {'name': 'Alessandro',
  'ticker': 'GOOG',
  'exchange': 'NASDAQ',
  'weight': 0.25},
 {'name': 'Alessandro', 'ticker': 'ZG', 'exchange': 'NASDAQ', 'weight': 0.35},
 {'name': 'Ivan', 'ticker': 'VST', 'exchange': 'NYSE', 'weight': 0.4},
 {'name': 'Ivan', 'ticker': 'EXPE', 'exchange': 'NASDAQ', 'weight': 0.3},
 {'name': 'Ivan', 'ticker': 'DGX', 'exchange': 'NYSE', 'weight': 0.3},
 {'name': 'Abhi', 'ticker': 'ASR', 'exchange': 'NYSE', 'weight': 0.25},
 {'name': 'Abhi', 'ticker': '0700', 'exchange': 'HKG', 'weight': 0.25},
 {'name': 'Abhi', 'ticker': 'SMCI', 'exchange': 'NASDAQ', 'weight': 0.25},
 {'name': 'Abhi',

In [115]:
# Define the start date for data download (from 2000)

FIXED_DATE = '2025-03-01'
INITIAL_VALUE_USD = 1e5

START_DATE = '2000-01-01'
END_DATE = datetime.now().strftime('%Y-%m-%d')

In [116]:
# check one ticker in Yahoo Finance
ticker = 'SCAN.V'
df_test = yf.download(ticker, start=START_DATE, end=END_DATE)

df_test.tail(5)

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,SCAN.V,SCAN.V,SCAN.V,SCAN.V,SCAN.V
Date,,,,,
2025-02-26,1.97,2.01,1.91,1.97,191149
2025-02-27,1.97,1.99,1.87,1.98,99640
2025-02-28,1.64,1.70,1.53,1.70,203504
2025-03-03,1.58,1.65,1.56,1.65,27569
2025-03-04,1.37,1.59,1.37,1.51,92550


In [117]:
# Get unique tickers and track which ones are from Hong Kong
unique_tickers = []
hk_tickers = []  # Hong Kong stocks (HKD)
eur_tickers = []  # European stocks (EUR)
gbp_tickers = []  # UK stocks (GBP)
cad_tickers = []  # Canadian stocks (CAD)

for position in POSITIONS:
    ticker = position['ticker']
    exchange = position['exchange']
    
    # Format ticker based on exchange
    if exchange == 'NYSE' or exchange == 'NASDAQ':
        formatted_ticker = ticker
    elif exchange == 'HKG':
        formatted_ticker = f"{ticker}.HK"  # Hong Kong stocks use .HK suffix
        hk_tickers.append(formatted_ticker)
    elif exchange == 'EPA':
        formatted_ticker = f"{ticker}.PA"  # Paris stocks use .PA suffix
        eur_tickers.append(formatted_ticker)
    elif exchange == 'LON':
        formatted_ticker = f"{ticker}.L"  # London stocks use .L suffix
        gbp_tickers.append(formatted_ticker)
    elif exchange == 'TSE':
        formatted_ticker = f"{ticker}-A.TO"  # Toronto stocks use .TO suffix (CTC-A is a special case)
        cad_tickers.append(formatted_ticker)
    elif exchange == 'CVE':
        formatted_ticker = f"{ticker}.V"  # .V according to the Yahoo Finance
        cad_tickers.append(formatted_ticker)
    elif exchange == 'XETRA':
        formatted_ticker = f"{ticker}.DE"  # .DE according to the Yahoo Finance (ETR/XETRA stock exchange)
        eur_tickers.append(formatted_ticker)
    else:
        formatted_ticker = ticker
        
    unique_tickers.append(formatted_ticker)

print(f"Downloading data for {len(unique_tickers)} unique tickers from {START_DATE} to {END_DATE}")

# Download historical data for all tickers
stock_data = yf.download(unique_tickers, start=START_DATE, end=END_DATE)

# We'll use the 'Close' prices for our calculations
prices_df = stock_data['Close'].copy()  # Create a copy to avoid SettingWithCopyWarning

# Download HKD to USD exchange rate data if we have Hong Kong stocks
if hk_tickers:
    print("\nDownloading HKD/USD exchange rate data for currency conversion...")
    # HKDUSD=X is the Yahoo Finance ticker for HKD to USD exchange rate
    exchange_rate_data = yf.download('HKDUSD=X', start=START_DATE, end=END_DATE)
    exchange_rate = exchange_rate_data['Close']
    
    # Convert Hong Kong stock prices from HKD to USD using the join approach
    for ticker in hk_tickers:
        print(f"\nProcessing {ticker} using join operation...")
        
        # Check if ticker exists in prices_df
        if ticker in prices_df.columns:
            # Get the HKD prices
            hkd_prices = prices_df[ticker]
        else:
            # Download the data directly
            print(f"Ticker {ticker} not found in original data, downloading directly...")
            single_stock_data = yf.download(ticker, start=START_DATE, end=END_DATE)
            
            if single_stock_data.empty:
                print(f"Error: Could not download data for {ticker}")
                continue
                
            hkd_prices = single_stock_data['Close']
        
        # Create a temporary DataFrame with both series properly indexed
        temp_df = pd.DataFrame({'HKD_Price': hkd_prices})
        temp_df['USD_Rate'] = exchange_rate  # This will align indices automatically
        
        # Calculate USD prices only where both values exist
        temp_df['USD_Price'] = temp_df['HKD_Price'] * temp_df['USD_Rate']
        
        # Print statistics
        print(f"Original HKD prices: {hkd_prices.notna().sum()} non-null values")
        print(f"Exchange rates: {exchange_rate.notna().sum()} non-null values")
        print(f"Converted USD prices: {temp_df['USD_Price'].notna().sum()} non-null values")
        
        # Add the converted prices to prices_df
        prices_df.loc[:, ticker] = temp_df['USD_Price']
        
        print(f"Successfully processed {ticker}")

# Display the first few rows of the data
print("\nFirst few rows of price data (all in USD):")
prices_df.tail(10)

[***********           22%                       ]  6 of 27 completed

[*********************100%***********************]  27 of 27 completed
[*********************100%***********************]  1 of 1 completed




Processing 0700.HK using join operation...
Original HKD prices: 5116 non-null values
Exchange rates: Ticker
HKDUSD=X    5975
dtype: int64 non-null values
Converted USD prices: 5089 non-null values
Successfully processed 0700.HK

Processing 0700.HK using join operation...
Original HKD prices: 5089 non-null values
Exchange rates: Ticker
HKDUSD=X    5975
dtype: int64 non-null values
Converted USD prices: 5089 non-null values
Successfully processed 0700.HK

First few rows of price data (all in USD):


Ticker,0700.HK,ALV.DE,ASR,BRO,DBK.DE,DGX,DUOL,EXPE,FLUT,GOOG,...,RHM.DE,RR.L,RTX,SCAN.V,SHEL.L,SMCI,TAK,TSM,VST,ZG
Date,,,,,,,,,,,,,,,,,,,,,
2025-02-19,8.236090,320.000000,283.880005,112.309998,19.486000,171.750000,432.519989,206.520004,291.750000,187.130005,...,932.799988,642.599976,124.778488,1.91,2686.0,60.250000,13.74,200.949997,169.350006,76.650002
2025-02-20,8.047015,319.500000,287.190002,111.650002,19.370001,172.139999,421.779999,205.139999,287.739990,186.639999,...,889.599976,618.400024,124.479996,2.20,2684.0,59.270000,13.86,200.100006,163.179993,75.449997
2025-02-21,8.551071,320.200012,273.510010,111.709999,19.642000,172.240005,385.350006,196.809998,272.609985,181.580002,...,894.200012,611.400024,123.290001,2.00,2637.5,56.070000,14.06,198.240005,150.440002,72.040001
2025-02-24,8.236480,323.100006,270.399994,112.599998,19.719999,174.059998,389.899994,199.970001,270.260010,181.190002,...,951.400024,606.200012,125.430000,1.89,2633.5,51.610001,14.08,191.649994,142.759995,72.650002
2025-02-25,8.022059,326.000000,269.890015,114.769997,20.105000,175.410004,385.809998,199.759995,260.489990,177.369995,...,952.799988,615.000000,127.339996,1.98,2629.5,45.540001,14.36,189.369995,137.919998,74.720001
2025-02-26,8.299827,334.700012,273.869995,114.790001,20.900000,173.460007,386.559998,199.539993,273.220001,174.699997,...,970.000000,631.000000,127.309998,1.97,2647.0,51.110001,14.34,194.619995,148.190002,75.699997
2025-02-27,8.199294,333.299988,264.859985,116.500000,20.500000,171.550003,375.779999,193.550003,271.769989,170.210007,...,1001.000000,731.599976,130.520004,1.97,2652.0,42.950001,14.34,181.089996,130.009995,73.860001
2025-02-28,7.913520,330.299988,270.820007,118.540001,20.725000,172.899994,312.070007,197.960007,280.589996,172.220001,...,1006.500000,744.000000,132.990005,1.64,2636.0,41.459999,14.41,180.529999,133.660004,74.389999
2025-03-03,7.987691,340.299988,264.989990,119.870003,20.950001,176.949997,295.230011,193.500000,271.200012,168.660004,...,1144.500000,776.799988,133.830002,1.58,2656.5,36.070000,14.84,172.970001,123.949997,72.040001


In [118]:
# OTHER CURRENCIES : EUR, GBP, CAD
# -------------------------------
# Download EUR to USD exchange rate data if we have European stocks
if eur_tickers:
    print("\nDownloading EUR/USD exchange rate data for currency conversion...")
    eur_usd_data = yf.download('EURUSD=X', start=START_DATE, end=END_DATE)
    eur_usd_rate = eur_usd_data['Close']
    print(f"Downloaded {len(eur_usd_rate)} days of EUR/USD rates")

# Download GBP to USD exchange rate data if we have UK stocks
if gbp_tickers:
    print("\nDownloading GBP/USD exchange rate data for currency conversion...")
    gbp_usd_data = yf.download('GBPUSD=X', start=START_DATE, end=END_DATE)
    gbp_usd_rate = gbp_usd_data['Close']
    print(f"Downloaded {len(gbp_usd_rate)} days of GBP/USD rates")

# Download CAD to USD exchange rate data if we have Canadian stocks
if cad_tickers:
    print("\nDownloading CAD/USD exchange rate data for currency conversion...")
    cad_usd_data = yf.download('CADUSD=X', start=START_DATE, end=END_DATE)
    cad_usd_rate = cad_usd_data['Close']
    print(f"Downloaded {len(cad_usd_rate)} days of CAD/USD rates")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



Downloaded 5517 days of EUR/USD rates

Downloaded 5529 days of GBP/USD rates

Downloaded 5585 days of CAD/USD rates


In [119]:
# Convert EUR stocks to USD using the join approach
for ticker in eur_tickers:
    print(f"\nProcessing {ticker} (EUR) using join operation...")
    
    # Check if ticker exists in prices_df
    if ticker in prices_df.columns:
        eur_prices = prices_df[ticker]
    else:
        # Download the data directly
        print(f"Ticker {ticker} not found in original data, downloading directly...")
        stock_data = yf.download(ticker, start=START_DATE, end=END_DATE)
        
        if stock_data.empty:
            print(f"Error: Could not download data for {ticker}")
            continue
            
        eur_prices = stock_data['Close']
    
    # Create a temporary DataFrame with both series properly indexed
    temp_df = pd.DataFrame({'EUR_Price': eur_prices})
    temp_df['USD_Rate'] = eur_usd_rate  # This will align indices automatically
    
    # Calculate USD prices only where both values exist
    temp_df['USD_Price'] = temp_df['EUR_Price'] * temp_df['USD_Rate']
    
    # Print statistics
    print(f"Original EUR prices: {eur_prices.notna().sum()} non-null values")
    print(f"Exchange rates: {eur_usd_rate.notna().sum()} non-null values")
    print(f"Converted USD prices: {temp_df['USD_Price'].notna().sum()} non-null values")
    
    # Add the converted prices to prices_df
    prices_df.loc[:, ticker] = temp_df['USD_Price']
    
    print(f"Successfully processed {ticker}")

# Convert GBP stocks to USD using the join approach
for ticker in gbp_tickers:
    print(f"\nProcessing {ticker} (GBP) using join operation...")
    
    # Check if ticker exists in prices_df
    if ticker in prices_df.columns:
        gbp_prices = prices_df[ticker]
    else:
        # Download the data directly
        print(f"Ticker {ticker} not found in original data, downloading directly...")
        stock_data = yf.download(ticker, start=START_DATE, end=END_DATE)
        
        if stock_data.empty:
            print(f"Error: Could not download data for {ticker}")
            continue
            
        gbp_prices = stock_data['Close']
    
    # Create a temporary DataFrame with both series properly indexed
    temp_df = pd.DataFrame({'GBP_Price': gbp_prices})
    temp_df['USD_Rate'] = gbp_usd_rate  # This will align indices automatically
    
    # Calculate USD prices only where both values exist
    temp_df['USD_Price'] = temp_df['GBP_Price'] * temp_df['USD_Rate']
    
    # Print statistics
    print(f"Original GBP prices: {gbp_prices.notna().sum()} non-null values")
    print(f"Exchange rates: {gbp_usd_rate.notna().sum()} non-null values")
    print(f"Converted USD prices: {temp_df['USD_Price'].notna().sum()} non-null values")
    
    # Add the converted prices to prices_df
    prices_df.loc[:, ticker] = temp_df['USD_Price']
    
    print(f"Successfully processed {ticker}")

# Convert CAD stocks to USD using the join approach
for ticker in cad_tickers:
    print(f"\nProcessing {ticker} (CAD) using join operation...")
    
    # Check if ticker exists in prices_df
    if ticker in prices_df.columns:
        cad_prices = prices_df[ticker]
    else:
        # Download the data directly
        print(f"Ticker {ticker} not found in original data, downloading directly...")
        stock_data = yf.download(ticker, start=START_DATE, end=END_DATE)
        
        if stock_data.empty:
            print(f"Error: Could not download data for {ticker}")
            continue
            
        cad_prices = stock_data['Close']
    
    # Create a temporary DataFrame with both series properly indexed
    temp_df = pd.DataFrame({'CAD_Price': cad_prices})
    temp_df['USD_Rate'] = cad_usd_rate  # This will align indices automatically
    
    # Calculate USD prices only where both values exist
    temp_df['USD_Price'] = temp_df['CAD_Price'] * temp_df['USD_Rate']
    
    # Print statistics
    print(f"Original CAD prices: {cad_prices.notna().sum()} non-null values")
    print(f"Exchange rates: {cad_usd_rate.notna().sum()} non-null values")
    print(f"Converted USD prices: {temp_df['USD_Price'].notna().sum()} non-null values")
    
    # Add the converted prices to prices_df
    prices_df.loc[:, ticker] = temp_df['USD_Price']
    
    print(f"Successfully processed {ticker}")

# --------------------------------------------


Processing ALV.DE (EUR) using join operation...
Original EUR prices: 6435 non-null values
Exchange rates: Ticker
EURUSD=X    5517
dtype: int64 non-null values
Converted USD prices: 5387 non-null values
Successfully processed ALV.DE

Processing DBK.DE (EUR) using join operation...
Original EUR prices: 6435 non-null values
Exchange rates: Ticker
EURUSD=X    5517
dtype: int64 non-null values
Converted USD prices: 5387 non-null values
Successfully processed DBK.DE

Processing RHM.DE (EUR) using join operation...
Original EUR prices: 6435 non-null values
Exchange rates: Ticker
EURUSD=X    5517
dtype: int64 non-null values
Converted USD prices: 5387 non-null values
Successfully processed RHM.DE

Processing RR.L (GBP) using join operation...
Original GBP prices: 6409 non-null values
Exchange rates: Ticker
GBPUSD=X    5529
dtype: int64 non-null values
Converted USD prices: 5372 non-null values
Successfully processed RR.L

Processing SHEL.L (GBP) using join operation...
Original GBP prices: 64

In [120]:
# Check that each person's weights sum to 100%
person_weights = {}

for position in POSITIONS:
    name = position['name']
    weight = position['weight']
    
    if name not in person_weights:
        person_weights[name] = 0
    
    person_weights[name] += weight

# Create a DataFrame for visualization
weights_df = pd.DataFrame({
    'Person': list(person_weights.keys()),
    'Total Weight': list(person_weights.values())
})

# Add a status column
weights_df['Status'] = weights_df['Total Weight'].apply(lambda x: 'OK' if abs(x - 1.0) < 0.01 else 'ERROR - Not 100%')

# Display the results
print("Weight sum for each person:")
for name, total_weight in person_weights.items():
    print(f"{name}: {total_weight:.2f} ({'OK' if abs(total_weight - 1.0) < 0.01 else 'ERROR - Not 100%'})")

# Create a bar chart to visualize the weights
fig = px.bar(
    weights_df, 
    x='Person', 
    y='Total Weight',
    color='Status',
    title='Portfolio Weight Distribution by Person',
    text='Total Weight',
    color_discrete_map={'OK': 'green', 'ERROR - Not 100%': 'red'}
)

# Add a horizontal line at 1.0 (100%)
fig.add_shape(
    type="line",
    x0=-0.5,
    y0=1.0,
    x1=len(weights_df)-0.5,
    y1=1.0,
    line=dict(color="black", width=2, dash="dash")
)

# Format the y-axis as percentage
fig.update_layout(
    yaxis=dict(title="Total Weight", tickformat=".0%"),
    template='presentation'
)

# Show the figure
fig.show()

Weight sum for each person:
Apoorva: 1.00 (OK)
Alessandro: 1.00 (OK)
Ivan: 1.00 (OK)
Abhi: 1.00 (OK)
Conor: 1.00 (OK)
Diarbhail: 1.00 (OK)
Radu: 1.00 (OK)
Silvia: 1.00 (OK)


In [121]:
# Allocate the initial investment based on weights
portfolio_allocations = []

for position in POSITIONS:
    name = position['name']
    ticker = position['ticker']
    exchange = position['exchange']
    weight = position['weight']
    
    # Format ticker based on exchange
    if exchange == 'NYSE' or exchange == 'NASDAQ':
        formatted_ticker = ticker
    elif exchange == 'HKG':
        formatted_ticker = f"{ticker}.HK"  # Hong Kong stocks use .HK suffix
    elif exchange == 'EPA':
        formatted_ticker = f"{ticker}.PA"  # Paris stocks use .PA suffix
    elif exchange == 'LON':
        formatted_ticker = f"{ticker}.L"  # London stocks use .L suffix
    elif exchange == 'TSE':
        formatted_ticker = f"{ticker}-A.TO"  # Toronto stocks use .TO suffix (CTC-A is a special case)
    elif exchange == 'CVE':
        formatted_ticker = f"{ticker}.V"  # .V according to the Yahoo Finance
    elif exchange == 'XETRA':
        formatted_ticker = f"{ticker}.DE"  # .V according to the Yahoo Finance
    else:
        formatted_ticker = ticker
    
    # Calculate allocation amount
    # Normalize by the person's total weight to ensure 100% allocation
    allocation_usd = INITIAL_VALUE_USD * (weight / person_weights[name])
    
    portfolio_allocations.append({
        'name': name,
        'ticker': formatted_ticker,
        'weight': weight,
        'normalized_weight': weight / person_weights[name],
        'allocation_usd': allocation_usd
    })

# Create a DataFrame for better visualization
allocations_df = pd.DataFrame(portfolio_allocations)
print("Initial portfolio allocations:")
allocations_df

Initial portfolio allocations:


,name,ticker,weight,normalized_weight,allocation_usd
0,Apoorva,TSM,0.40,0.40,40000.0
1,Apoorva,GOOGL,0.10,0.10,10000.0
2,Apoorva,TAK,0.25,0.25,25000.0
3,Apoorva,0700.HK,0.25,0.25,25000.0
4,Alessandro,DUOL,0.40,0.40,40000.0
5,Alessandro,GOOG,0.25,0.25,25000.0
6,Alessandro,ZG,0.35,0.35,35000.0
7,Ivan,VST,0.40,0.40,40000.0
8,Ivan,EXPE,0.30,0.30,30000.0
9,Ivan,DGX,0.30,0.30,30000.0


In [122]:
# Visualize the allocations by person
fig = px.treemap(
    allocations_df,
    path=['name', 'ticker'],
    values='allocation_usd',
    title='Portfolio Allocations by Person and Stock',
    color='allocation_usd',
    color_continuous_scale='Viridis',
    hover_data=['normalized_weight']
)

fig.update_layout(template='presentation')
fig.update_traces(texttemplate="%{label}<br>$%{value:,.0f}<br>%{customdata[0]:.1%}", textposition="middle center")
fig.show()

In [123]:
# Let's check if we have data for all tickers
print("Checking data availability for all tickers:")
for ticker in unique_tickers:
    if ticker in prices_df.columns:
        data_count = prices_df[ticker].count()
        print(f"{ticker}: {data_count} data points available")
    else:
        print(f"{ticker}: No data available")

# If 0700.HK is missing, try to download it directly
if '0700.HK' not in prices_df.columns or prices_df['0700.HK'].isna().all():
    print("\nAttempting to download 0700.HK directly...")
    tencent_data = yf.download('0700.HK', start=START_DATE, end=END_DATE)
    
    if not tencent_data.empty:
        # Add to our main dataframe
        prices_df['0700.HK'] = tencent_data['Close']
        
        # Convert HKD to USD
        common_dates = prices_df.index.intersection(exchange_rate.index)
        prices_df.loc[common_dates, '0700.HK'] = prices_df.loc[common_dates, '0700.HK'] * exchange_rate.loc[common_dates]
        print(f"Successfully downloaded and converted 0700.HK prices from HKD to USD")
        
        # Check data again
        data_count = prices_df['0700.HK'].count()
        print(f"0700.HK: {data_count} data points available after direct download")
    else:
        print(f"Error: Could not download data for 0700.HK")

Checking data availability for all tickers:
TSM: 6330 data points available
GOOGL: 5168 data points available
TAK: 3814 data points available
0700.HK: 5089 data points available
DUOL: 904 data points available
GOOG: 5168 data points available
ZG: 3426 data points available
VST: 2114 data points available
EXPE: 4936 data points available
DGX: 6330 data points available
ASR: 6143 data points available
0700.HK: 5089 data points available
SMCI: 4512 data points available
BRO: 6330 data points available
PFE: 6330 data points available
RR.L: 5372 data points available
MCD: 6330 data points available
SHEL.L: 5372 data points available
ALV.DE: 5387 data points available
FLUT: 5639 data points available
NVDA: 6330 data points available
SCAN.V: 2935 data points available
MP: 1181 data points available
DBK.DE: 5387 data points available
RTX: 6330 data points available
RHM.DE: 5387 data points available
NVO: 6330 data points available
NVDA: 6330 data points available
OPRA: 1659 data points availab

In [124]:
# Calculate the number of shares to buy based on the price at FIXED_DATE
# First, find the closest date in our data to FIXED_DATE
closest_date = prices_df.index[prices_df.index <= FIXED_DATE][-1]
print(f"Using prices from {closest_date} for allocation calculations")

# Get prices on that date
prices_on_date = prices_df.loc[closest_date]

# Calculate shares and actual allocation
portfolio = []

for alloc in portfolio_allocations:
    ticker = alloc['ticker']
    allocation_usd = alloc['allocation_usd']
    
    # Get the price on the fixed date
    if ticker in prices_on_date and not pd.isna(prices_on_date[ticker]):
        price = prices_on_date[ticker]
        shares = allocation_usd / price
        actual_allocation = shares * price
        
        portfolio.append({
            'name': alloc['name'],
            'ticker': ticker,
            'shares': shares,
            'price_at_start': price,
            'allocation_usd': actual_allocation
        })
    else:
        print(f"Warning: No price data available for {ticker} on {closest_date}")

# Create a DataFrame for the portfolio
portfolio_df = pd.DataFrame(portfolio)
print("\nFinal portfolio with share allocations:")
portfolio_df

Using prices from 2025-02-28 00:00:00 for allocation calculations

Final portfolio with share allocations:


,name,ticker,shares,price_at_start,allocation_usd
0,Apoorva,TSM,221.569824,180.529999,40000.0
1,Apoorva,GOOGL,58.726803,170.279999,10000.0
2,Apoorva,TAK,1734.906333,14.410000,25000.0
3,Apoorva,0700.HK,3159.150467,7.913520,25000.0
4,Alessandro,DUOL,128.176368,312.070007,40000.0
5,Alessandro,GOOG,145.163162,172.220001,25000.0
6,Alessandro,ZG,470.493350,74.389999,35000.0
7,Ivan,VST,299.266788,133.660004,40000.0
8,Ivan,EXPE,151.545762,197.960007,30000.0
9,Ivan,DGX,173.510706,172.899994,30000.0


In [125]:
# Calculate daily portfolio value from FIXED_DATE onwards
# Filter prices to dates >= FIXED_DATE
prices_since_start = prices_df[prices_df.index >= closest_date]

# Initialize a DataFrame to store portfolio values
portfolio_values = pd.DataFrame(index=prices_since_start.index)

# Calculate daily value for each person
for name in set(p['name'] for p in portfolio):
    person_portfolio = [p for p in portfolio if p['name'] == name]
    daily_values = pd.Series(0, index=prices_since_start.index)
    
    for position in person_portfolio:
        ticker = position['ticker']
        shares = position['shares']
        
        if ticker in prices_since_start.columns:
            # Calculate daily value of this position
            position_value = prices_since_start[ticker] * shares
            daily_values += position_value
    
    portfolio_values[name] = daily_values

# Display the first few rows
print("Daily portfolio values since start date (USD):")
portfolio_values.head()

Daily portfolio values since start date (USD):


,Diarbhail,Apoorva,Radu,Alessandro,Abhi,Conor,Ivan,Silvia
Date,,,,,,,,
2025-02-28,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
2025-03-03,94791.497118,99113.223997,102926.952744,96219.071280,96726.512181,101412.143512,97120.941252,94715.531834
2025-03-04,90369.836466,101294.164652,98299.076802,97006.902673,98762.632808,100811.901380,96951.719281,94125.549685


In [126]:
# Plot the portfolio values over time using Plotly
fig = go.Figure()

# Plot individual portfolios
for name in portfolio_values.columns:
    fig.add_trace(go.Scatter(
        x=portfolio_values.index,
        y=portfolio_values[name],
        name=f"{name}'s Portfolio",
        mode='lines'
    ))

# Add a horizontal line for initial investment
fig.add_shape(
    type="line",
    x0=portfolio_values.index[0],
    y0=INITIAL_VALUE_USD,
    x1=portfolio_values.index[-1],
    y1=INITIAL_VALUE_USD,
    line=dict(color="red", width=2, dash="dash")
)

# Update layout
fig.update_layout(
    title='Portfolio Performance Since Start Date',
    xaxis_title='Date',
    yaxis_title='Portfolio Value (USD)',
    template='presentation',
    hovermode='x unified'
)

# Show the figure
fig.show()

In [127]:
# Calculate percentage change from start date
pct_change = portfolio_values.copy()
for col in pct_change.columns:
    pct_change[col] = (pct_change[col] / pct_change[col].iloc[0] - 1) * 100

# Plot the percentage change
fig = go.Figure()

# Plot individual portfolios
for name in pct_change.columns:
    fig.add_trace(go.Scatter(
        x=pct_change.index,
        y=pct_change[name],
        name=f"{name}'s Portfolio",
        mode='lines'
    ))

# Add a horizontal line at 0%
fig.add_shape(
    type="line",
    x0=pct_change.index[0],
    y0=0,
    x1=pct_change.index[-1],
    y1=0,
    line=dict(color="red", width=2, dash="dash")
)

# Update layout
fig.update_layout(
    title='Portfolio Performance (% Change) Since Start Date',
    xaxis_title='Date',
    yaxis_title='Percentage Change (%)',
    template='presentation',
    hovermode='x unified'
)

# Show the figure
fig.show()

In [128]:
# Calculate performance metrics
initial_value = portfolio_values.iloc[0]
final_value = portfolio_values.iloc[-1]

# Calculate returns
total_return = (final_value / initial_value - 1) * 100

# Calculate annualized returns
days = (portfolio_values.index[-1] - portfolio_values.index[0]).days
years = days / 365.25
annualized_return = ((final_value / initial_value) ** (1 / years) - 1) * 100

# Create a DataFrame for the results
performance_df = pd.DataFrame({
    'Initial Value': initial_value,
    'Final Value': final_value,
    'Total Return (%)': total_return,
    'Annualized Return (%)': annualized_return
})

print("Performance Summary:")
performance_df

Performance Summary:


,Initial Value,Final Value,Total Return (%),Annualized Return (%)
Diarbhail,100000.0,90369.836466,-9.630164,-99.990353
Apoorva,100000.0,101294.164652,1.294165,223.540025
Radu,100000.0,98299.076802,-1.700923,-79.122877
Alessandro,100000.0,97006.902673,-2.993097,-93.763859
Abhi,100000.0,98762.632808,-1.237367,-67.919424
Conor,100000.0,100811.901380,0.811901,109.253040
Ivan,100000.0,96951.719281,-3.048281,-94.079608
Silvia,100000.0,94125.549685,-5.874450,-99.602654


In [129]:
# Create a bar chart to visualize the total returns
performance_data = pd.DataFrame({
    'Portfolio': total_return.index,
    'Total Return (%)': total_return.values
})

# Sort the data to make the visualization more informative (optional)
performance_data = performance_data.sort_values('Total Return (%)', ascending=False)

# Create the figure with increased size
fig = px.bar(
    performance_data,
    x='Portfolio',
    y='Total Return (%)',
    title='Portfolio Returns by Competitor',
    color='Total Return (%)',
    color_continuous_scale=['red', 'yellow', 'green'],
    text='Total Return (%)'
)

# Add a horizontal line at 0%
fig.add_shape(
    type="line",
    x0=-0.5,
    y0=0,
    x1=len(performance_data)-0.5,
    y1=0,
    line=dict(color="black", width=2, dash="dash")
)

# Format the text labels and ensure they're visible
fig.update_traces(
    texttemplate='%{text:.1f}%', 
    textposition='outside',
    textfont=dict(size=14)  # Increase text size
)

# Calculate the y-axis range with extra padding to ensure text is visible
y_min = min(performance_data['Total Return (%)']) * 1.2 if min(performance_data['Total Return (%)']) < 0 else min(performance_data['Total Return (%)']) - 2
y_max = max(performance_data['Total Return (%)']) * 1.2 if max(performance_data['Total Return (%)']) > 0 else max(performance_data['Total Return (%)']) + 2

# Update layout with increased size, margins, and adjusted y-axis
fig.update_layout(
    template='presentation',
    yaxis=dict(
        title='Total Return (%)',
        range=[y_min, y_max]  # Ensure text labels are visible
    ),
    width=900,  # Increase width
    height=600,  # Increase height
    margin=dict(l=80, r=80, t=100, b=80),  # Increase margins
    title=dict(
        text='Portfolio Returns by Competitor',
        font=dict(size=24),  # Larger title
        y=0.95  # Position title higher
    )
)

# Show the figure
fig.show()